<a href="https://colab.research.google.com/github/ucfilho/raianars_adjust_RTC/blob/master/RTC_adjust_v_03_mar_05_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x,cols): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x):
  global Trocas,D  
  Ind_I=np.zeros((Trocas))
  Ind_J=np.zeros((Trocas))
  deltaT=np.zeros((Trocas))
  for i in range(D):
    for j in range(Trocas):
      Ind_I[j]=round(x[i,j])
      Ind_J[j]=round(x[i,j+Trocas])
      deltaT[j]=x[i,j+2*Trocas] 
  return Ind_I,Ind_J,deltaT

In [0]:
#def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial):
  global Trocas
  # cols=len(mCp)
  # i represent hot stream  and j represent cold stream 
  # k represent sequential heat exchanges
  cargaTermica=np.zeros((Trocas))
  contador=0
  cont_uso=0
  tempAtual=np.copy(tempInicial)
  erro=0 # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for k in range(Trocas):
    i=int(round(Ind_I[k]))
    j=int(round(Ind_J[k]))
    #print(mCp[i],deltaT[k])
    #print('==============')
    #print(deltaT)
    #print(mCp)
    #print(i)
    #print(j)
    cargaTermica[k]=mCp[i]*deltaT[k]
    tempSaidaQ=tempInicial[i]+deltaT[k]
    tempSaidaF=-cargaTermica[k]/(mCp[j])+tempAtual[j]
    flag=0 # flag : indica que nao entra no teste logico
    # usei flag no lugar q ifs pq a identacao ficou melhor assim
    if(tempAtual[i] > tempAtual[j]): #Tquente maior Tfria: vaores atuais
      if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
        if(contador < cols):
          flag=1
          
    if(flag>0):
      tempAtual[i]=tempSaidaQ
      tempAtual[j]=tempSaidaF
    else:
      erro=1


  return tempAtual,cargaTermica,erro

In [0]:
def Area(U,tempAtual,tempInicial,Ind_I,Ind_J,deltaT):
  global Trocas,mCp
  
  Area=[]

  for k in range(Trocas):
    i=int(Ind_I[k])
    j=int(Ind_J[k])
    Carga=mCp[k]*deltaT[k]
    tempSaidaF=tempAtual[j]
    tempSaidaQ=tempAtual[i]
    deltaT1=tempInicial[i]-tempSaidaF
    deltaT2=tempSaidaQ-tempInicial[j]
    try:
      A=(deltaT1-deltaT2)
      B=np.log((deltaT1/deltaT2))
      deltaTMediaLog=A/B
    except:
      deltaTMediaLog=1e99
 
    if(deltaTMediaLog==1e99):
      AreaCalc=1e99
    else:
      AreaCalc=abs(Carga/(U*deltaTMediaLog))

    if(tempAtual[i]==tempInicial[i]):
          AreaCalc=0

    if(tempAtual[j]==tempInicial[j]):
      AreaCalc=0
          
    Area.append(AreaCalc)

  return Area

In [0]:
def Utilidades(tempAlvo,tempAtual,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353
  areaTrocador=[]

  for j in range(cols): 
    delta=tempAlvo[j]-tempAtual[j]
    if(delta>0):#usa UQ
      cargaUQ=cargaUQ+mCp[j]*delta
      deltaT1=(TUQin-tempAlvo[j])
      deltaT2=(TUQout-tempAtual[j])

      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99

      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

    elif(delta==0):#nao usa UF e UQ
      deltaTMediaLog=0
      areaTrocador.append(0) 

    else:#usa UF
      deltaT1=(tempAtual[j]-TUFout)
      deltaT2=(tempAlvo[j]-TUFin)     
      cargaUF=cargaUF+mCp[j]*(-delta)

      #UF -> água resfriada -> entra a 303K e sai a 353K
      try:
        A=deltaT1-deltaT2
        B=np.log(deltaT1/deltaT2)
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      
      if(deltaTMediaLog==1e99):
        areaTrocador.append(1e99)
      else:
        areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

  return areaTrocador, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(areaTrocadores,areaUtilidades,cargaUF,cargaUQ):
  global coefCustoUQ,coefCustoUF
  costTrocadores=0
  for it in areaTrocadores:
    for wii in areaUtilidades:
      if(it<1e99):
        if(wii<1e99):
          costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
        else:
          costTrocadores=1e99
          break
        
  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF

  if(costTrocadores==1e99):
    cost=1e99
  else:
    cost=costUQ+costUF+costTrocadores
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  # global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  


  cols=len(mCp)
  #tempInicial=np.copy(tempAtual) 
  Ind_I,Ind_J,deltaT=buildMatriz(x) # transforma x em matriz logica e troca

  tempAtual,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial)
  areaTrocadores =Area(U,tempAtual,tempInicial,Ind_I,Ind_J,deltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr = np.where(np.isnan(cost), 1e99, cost)

  if(Erro>0):
    funr=1e99


  return funr

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=5000 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
MIN=[-300,-300,-300,-300,-300,-300,0,0,0,0,0,0]
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [0]:
#MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
#MIN=[-300,-300,-300,-300,-300,-300,0.3,0.3,0.3,0.3,0.3,0.3]
Trocas=4

D=5

cols=Trocas*3 # tres linhas i,j,deltaT
MAX=np.zeros((cols))
MIN=np.zeros((cols))

for k in range(cols):
  if(k>(Trocas*2-1)):
    MIN[k]=-100
  else:
    MAX[k]=(Trocas-1)

x=np.zeros((D,cols))

for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

Ind_I=np.zeros((Trocas))
Ind_J=np.zeros((Trocas))
deltaT=np.zeros((Trocas))
for i in range(D):
  for j in range(Trocas):
    Ind_I[j]=round(x[i,j])
    Ind_J[j]=round(x[i,j+Trocas])
    deltaT[j]=x[i,j+2*Trocas]





In [13]:

Custo=FUN(x)
print(Custo)


------------
0
3
------------
0
0
------------
2
1
------------
3
2
1e+99


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in long_scalars


In [14]:
print(FUN(x))

------------
0
3
------------
0
0
------------
2
1
------------
3
2
1e+99


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in long_scalars


In [0]:
cols=len(mCp)
UsoFuncao=[]
x=[-99.214722,59.99994,146.225,4.2108538,15.99999,0, 0.9500049,1, 1,0.9103451, 1,0.4786627]
matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
tempAtual,Carga,Erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
print('estou aqui');print(areaUtilidade);print(areaTrocadores)
custoTotal=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo total');print(custoTotal);
cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
print('Custo processo');print(cost);
print('cont_uso');print(UsoFuncao)

In [0]:
UsoFuncao=[]
fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n'%(ITE,NPAR));print(GBEST)
print('Valor da função f(gbest)=%f \n'%BEST)
print('matriz grafico');print(matrizGrafico)
print('cont_uso');print(UsoFuncao)

In [0]:
cols=len(mCp)
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
tempAtual,cargaTermica,erro=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
areaTrocadores =Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT)

print('temperatura inicial');print(tempInicial)
print('temperatura alvo');print(tempAlvo)
print('temperatura atual');print(tempAtual)
print('Area trocadores');print(areaTrocadores)

In [0]:
print(cols)
print(GBEST)

In [0]:
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
print('Matriz de Trocas Térmicas \n')
print(matrizDeltaT)
print('Matriz Binária \n')
print(matrizBinaria)